# Notebook 4: Ajuste del Modelo SARIMAX, Pronósticos y Validación del Modelo
**Proyecto:** Análisis SARIMAX - Starbucks Corporation (SBUX)  
**Investigador:** Frankli Zeña Zeña (UNI)
___   

En este notebook construiremos el modelo SARIMAX óptimo para predecir el precio de cierre (`Close`) de Starbucks. 
Nuestro enfoque será iterativo:
1. **Búsqueda Automática:** Usaremos `Auto-ARIMA` para encontrar los hiperparámetros $(p, d, q) \times (P, D, Q)_m$.
2. **Evaluación de Exógenas:** Analizaremos el *Summary* del modelo para evaluar la significancia estadística ($P-valor$) de nuestras variables financieras y dummies.
3. **Poda (Feature Selection):** Eliminaremos las variables que no aporten poder predictivo ($P > 0.05$) para evitar el sobreajuste (overfitting).

## Carga y Split de Datos

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
ruta_archivo = '../data/transformed/sbux_master_sarimax.csv'
df = pd.read_csv(ruta_archivo, index_col='Fecha', parse_dates=True).dropna()
df

,Date,Adj Close,Volume,Vol_Avg_20,Vol_Anomaly,Log_Return,Margen_Operativo_%,Revenue,choque_estructural,shock_extremo,earnings,riesgo_pais,shock_costos
Fecha,,,,,,,,,,,,,
2021-03-31,2021-03-31,97.421112,6478400,7244615.0,False,-0.009110,14.811039,6668.0,0,0,0,0,0
2021-04-01,2021-04-01,97.519173,5793000,7173675.0,False,0.001006,14.811039,6668.0,0,0,0,0,0
2021-04-05,2021-04-05,98.981331,6913100,7241335.0,False,0.014882,14.811039,6668.0,0,0,0,0,0
2021-04-06,2021-04-06,100.880363,6745200,7322620.0,False,0.019004,14.811039,6668.0,0,0,0,0,0
2021-04-07,2021-04-07,100.916023,5629600,7328555.0,False,0.000353,14.811039,6668.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-09,2026-02-09,98.345779,7150600,11434955.0,False,-0.004737,8.984274,9915.1,1,0,0,0,0
2026-02-10,2026-02-10,96.905067,8543500,11497655.0,False,-0.014758,8.984274,9915.1,1,0,0,0,0
2026-02-11,2026-02-11,98.484879,6949100,11567015.0,False,0.016171,8.984274,9915.1,1,0,0,0,0


In [21]:
split_idx = int(len(df) * 0.8)

In [22]:
df_train = df[:split_idx]
df_test = df[split_idx:]

In [23]:
target_col = 'Adj Close'

y = df[target_col]

exog_cols = [
    'Margen_Operativo_%', 'Revenue', 
    'choque_estructural', 'shock_extremo', 
    'earnings', 'riesgo_pais', 'shock_costos'
]

X = df[exog_cols]

In [24]:

y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]

print(f"Datos listos. Train: {len(y_train)} obs. | Test: {len(y_test)} obs.")

Datos listos. Train: 980 obs. | Test: 245 obs.


## Paso 1: Búsqueda del Modelo Óptimo (Auto-ARIMA)

Del Notebook 3 sabemos que la serie original requiere una primera diferencia ($d=1$) para ser estacionaria. Además, identificamos un ciclo estacional de $m=6$.
Inyectaremos nuestra matriz `X_train` al algoritmo para que busque la mejor combinación de rezagos autorregresivos y medias móviles que minimice el Criterio de Información de Akaike (AIC), para ello utilizaremos Auto-ARIMA.  
El algoritmo iterará sobre múltiples combinaciones de $p$, $q$, $P$ y $Q$ para minimizar el **Criterio de Información de Akaike (AIC)**. El modelo con el AIC más bajo será nuestro modelo óptimo.

In [25]:
# %pip install pmdarima
import pmdarima as pm

In [11]:
print("Iniciando búsqueda Auto-ARIMA con Variables Exógenas...")

# Ejecutamos el Auto-ARIMA sobre nuestro conjunto de Entrenamiento (y_train, X_train)
modelo_auto = pm.auto_arima(
    y=y_train, 
    X=X_train,               # ¡Aquí entran tus finanzas y dummies!
    start_p=0, start_q=0,    # Empezamos desde cero
    max_p=2, max_q=2,        # Límite máximo para p y q (para no sobreajustar)
    d=1,                     # Diferencia confirmada por Dickey-Fuller
    seasonal=True,           # Activamos la estacionalidad
    m=21,                     # Ciclo estacional de 6 periodos
    start_P=0, start_Q=0,
    max_P=2, max_Q=2,
    D=1,                     # Diferencia estacional (usualmente 1 si hay estacionalidad fuerte)
    trace=True,              # Imprime el proceso paso a paso
    error_action='ignore',  
    suppress_warnings=True, 
    stepwise=True            # Búsqueda inteligente (más rápida)
)

print("\n ¡Búsqueda completada!")
print()

Iniciando búsqueda Auto-ARIMA con Variables Exógenas...
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[21]             : AIC=4258.460, Time=2.22 sec
 ARIMA(1,1,0)(1,1,0)[21]             : AIC=3946.997, Time=25.81 sec
 ARIMA(0,1,1)(0,1,1)[21]             : AIC=inf, Time=22.90 sec
 ARIMA(1,1,0)(0,1,0)[21]             : AIC=4260.275, Time=2.03 sec
 ARIMA(1,1,0)(2,1,0)[21]             : AIC=3835.180, Time=62.83 sec
 ARIMA(1,1,0)(2,1,1)[21]             : AIC=inf, Time=69.38 sec
 ARIMA(1,1,0)(1,1,1)[21]             : AIC=inf, Time=21.45 sec
 ARIMA(0,1,0)(2,1,0)[21]             : AIC=3833.691, Time=45.97 sec
 ARIMA(0,1,0)(1,1,0)[21]             : AIC=3947.857, Time=18.31 sec
 ARIMA(0,1,0)(2,1,1)[21]             : AIC=inf, Time=52.57 sec
 ARIMA(0,1,0)(1,1,1)[21]             : AIC=inf, Time=20.59 sec
 ARIMA(0,1,1)(2,1,0)[21]             : AIC=3835.045, Time=43.86 sec
 ARIMA(1,1,1)(2,1,0)[21]             : AIC=3837.387, Time=65.49 sec
 ARIMA(0,1,0)(2,1,0)[21] intercept   : AIC=3

> Se observa que tanto los factores ``Margen_Operativo_%``, ``Revenue`` y ``riesgo_pais`` no tienen mayor influencia en la estimación del **Precio**, es preciso retirarlas

## Paso 2: Ajuste SARIMAX y Resumen Estadístico (Summary)

Entrenamos el modelo formalmente usando `statsmodels` con los hiperparámetros ganadores. 
El objetivo principal de esta celda es obtener la tabla de coeficientes y enfocarnos en la columna `P>|z|` para nuestras variables exógenas.
* **Hipótesis Nula ($H_0$):** La variable exógena no tiene efecto sobre el precio de Starbucks.
* Si $P \le 0.05$: Rechazamos $H_0$. La variable se queda (es estadísticamente significativa).
* Si $P > 0.05$: Aceptamos $H_0$. La variable es ruido y deberá ser eliminada en la siguiente iteración.

In [26]:
exog_cols_2 = [ 
    'choque_estructural', 'shock_extremo', 
    'earnings', 'shock_costos'
]

X_2 = df[exog_cols_2]

In [27]:
X_train_2, X_test_2 = X_2.iloc[:split_idx], X_2.iloc[split_idx:]

print(f"Datos listos. Train: {len(y_train)} obs. | Test: {len(y_test)} obs.")

Datos listos. Train: 980 obs. | Test: 245 obs.


In [34]:
orden_optimo = (0, 1, 0) 
orden_estacional = (2, 1, 0, 21) 

# Definimos el modelo maestro
modelo_sarimax = sm.tsa.statespace.SARIMAX(
    endog=y_train, 
    exog=X_train_2, 
    order=orden_optimo, 
    seasonal_order=orden_estacional,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustamos el modelo
resultados = modelo_sarimax.fit(disp=False)

# Imprimimos la radiografía del modelo
print(resultados.summary())

                                     SARIMAX Results                                      
Dep. Variable:                          Adj Close   No. Observations:                  980
Model:             SARIMAX(0, 1, 0)x(2, 1, 0, 21)   Log Likelihood               -1829.064
Date:                           lu., 16 feb. 2026   AIC                           3672.127
Time:                                    11:30:24   BIC                           3705.867
Sample:                                         0   HQIC                          3685.005
                                            - 980                                         
Covariance Type:                              opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
choque_estructural     4.5603      2.227      2.048      0.041       0.196       8.925
shock_extre

## Paso 3: Predicción (Forecasting) sobre el Conjunto Test

Una vez ajustado y depurado nuestro modelo SARIMAX, procederemos a generar las predicciones para el horizonte de tiempo que corresponde a nuestro conjunto `Test` (el 20% más reciente de los datos). 
Para ello, el algoritmo requiere que le suministremos la matriz de variables exógenas de ese periodo (`X_test`). Además de la predicción puntual, calcularemos los Intervalos de Confianza al 95%.

In [35]:
import plotly.graph_objects as go

In [37]:
# Generar la predicción indicando cuántos pasos a futuro (len de y_test) y entregando las variables exógenas de ese periodo temporal
forecast = resultados.get_forecast(steps=len(df_test), exog=X_test_2)
forecast_index = df_test.index
forecast_values = forecast.predicted_mean

# Intervalos de Confianza al 95%
conf_int = forecast.conf_int()

print(f"Total de días predichos: {len(forecast_values)}")

Total de días predichos: 245


### Paso 3.1: Visualización: Realidad vs. Predicción

Graficaremos el comportamiento real de la acción (`y_test`) frente a la proyección de nuestro modelo (`pred_media`), incluyendo la banda de los intervalos de confianza. Para dar contexto visual sin comprimir la gráfica, incluiremos solo los últimos 250 días del conjunto de entrenamiento.

In [38]:
fig = go.Figure()

# Datos reales (retornos reales)
fig.add_trace(go.Scatter(
    x=y.index,
    y=y,
    mode='lines',
    name='Datos reales',
    line=dict(color='black')
))

# Media
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast_values,
    mode='lines',
    name='Predicciones',
    line=dict(color='red')
))

# Banda superior intervalo confianza
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=conf_int.iloc[:, 1],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Banda inferior con relleno
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=conf_int.iloc[:, 0],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 105, 180, 0.4)',  # rosado similar a matplotlib
    line=dict(width=0),
    name='Intervalo Confianza 95%'
))

# Personalización
fig.update_layout(
    title='Predicciones SARIMAX',
    xaxis_title='Fecha',
    yaxis_title='Retorno Log',
    hovermode='x unified',
    template='plotly_white',
    height=1000
)

fig.show()


## Paso 4: Cálculo de Métricas de Error y Sesgo

Para cuantificar científicamente el rendimiento de nuestro modelo en el Camino 1 (Precio Absoluto), calcularemos los siguientes indicadores sobre el conjunto de prueba:
* **MSE (Mean Squared Error):** Penaliza fuertemente los errores grandes.
* **RMSE (Root Mean Squared Error):** El error promedio expresado en las mismas unidades que la variable (Dólares).
* **MAE (Mean Absolute Error):** La magnitud promedio de los errores sin considerar la dirección.
* **MAPE (Mean Absolute Percentage Error):** El error promedio en formato porcentual (ideal para comparar con otros modelos o activos).
* **Sesgo (Bias):** Promedio de los errores. Indica si el modelo tiende a sobreestimar (sesgo negativo) o subestimar (sesgo positivo) sistemáticamente.

In [48]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Calculamos los errores absolutos (Realidad - Predicción)
errores = y_test - forecast_values

# 1. Mean Squared Error (MSE)
mse = mean_squared_error(y_test, forecast_values)
# 2. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
# 3. Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, forecast_values)
# 4. Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, forecast_values) * 100
# 5. Sesgo (Bias) - Promedio aritmético de los errores
sesgo = errores.mean()

# Tabla de resultados
print("-" * 40)
print("================== INDICADORES DE RENDIMIENTO ==================")
print("-" * 40)
print(f"🔹 MSE   : {mse:.4f}")
print(f"🔹 RMSE  : ${rmse:.2f} USD (Error promedio por acción)")
print(f"🔹 MAE   : ${mae:.2f} USD (Desviación absoluta promedio)")
print(f"🔹 MAPE  : {mape:.2f}% (Error porcentual)")
print(f"🔹 Sesgo : {sesgo:.4f}")
print("-" * 40)

----------------------------------------
================== INDICADORES DE RENDIMIENTO ==================
----------------------------------------
🔹 MSE   : 6390.7592
🔹 RMSE  : $79.94 USD (Error promedio por acción)
🔹 MAE   : $73.27 USD (Desviación absoluta promedio)
🔹 MAPE  : 84.78% (Error porcentual)
🔹 Sesgo : nan
----------------------------------------


``Evaluación del SARIMAX predictivo... ``

````md
El intervalo de confianza es demasiado amplio (alta incertidumbre).  
La proyección alcista no es suficientemente confiable para tomar una decisión real de inversión.

Problemas observados
- Forecast con varianza muy alta.
- Posible mala especificación del modelo.
- No evidencia clara de superioridad frente a un random walk.

Recomendación
- Validar con RMSE/MAE vs modelo naive.
- Usarlo solo como herramienta complementaria, no como señal única.

## Paso 5: Pronóstico Walk-Forward

In [ ]:
# 1. Copiamos la historia inicial (Data de Entrenamiento)
historia_y = y_train.copy()       
historia_X = X_train_2.copy()    

predicciones = []
limites_inf = []
limites_sup = []

print("---------- Iniciando pronóstico Walk-Forward ----------")

# 2. Iteramos sobre cada día del Test
# for i in range(len(y_test)):
for i in range(50):
    
    if i % 10 == 0:
        print(f"Prediciendo día {i} de {len(y_test)}...")
        
    # A) Entrenamos el modelo con la historia hasta el día "de hoy"
    modelo_temp = sm.tsa.statespace.SARIMAX(
        endog=historia_y, 
        exog=historia_X, 
        order=orden_optimo,        
        seasonal_order=orden_estacional, 
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    res_temp = modelo_temp.fit(disp=False)
    
    # B) Predecimos SOLO 1 paso adelante (mañana) 
    exog_manana = X_test_2.iloc[[i]]
    fcst = res_temp.get_forecast(steps=1, exog=exog_manana)
    
    # C) Guardamos la predicción y el intervalo de confianza (estos sí van a la lista)
    predicciones.append(fcst.predicted_mean.iloc[0])
    limites_inf.append(fcst.conf_int().iloc[0, 0])
    limites_sup.append(fcst.conf_int().iloc[0, 1])
    
    # D) TRUCO CORREGIDO: Agregamos el valor real de "mañana" usando pd.concat
    y_manana = y_test.iloc[[i]] 
    
    # Concatenamos de forma segura por debajo (axis=0 por defecto)
    historia_y = pd.concat([historia_y, y_manana]) 
    historia_X = pd.concat([historia_X, exog_manana])

# 3. Formateamos los resultados para que funcionen con tus gráficas
forecast_index = y_test.index
forecast_values = pd.Series(predicciones, index=forecast_index)
conf_int = pd.DataFrame({'lower': limites_inf, 'upper': limites_sup}, index=forecast_index)

print(f"\n Total de días predichos (Walk-Forward): {len(forecast_values)}")